In [1]:
import pandas as pd
import requests
import csv
import json
from collections import OrderedDict
from heapq import nlargest
from heapq import nsmallest

### Collect and filter dataset for useful values and assign column names

In [2]:
data = pd.read_excel("DDW-0000C-08.xlsx")
data.head(7)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,C-8 EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE - 2011,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,Table,State,Distt.,Area Name,Total/,Age-group,Total,NaN,NaN,Illiterate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name,Code,Code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,...,Technical diploma or certificate,NaN,NaN,Graduate & above,NaN,NaN,Unclassified,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Urban/,NaN,NaN,NaN,NaN,NaN,...,not equal to degree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Persons,Males,Females,Persons,...,Persons,Males,Females,Persons,Males,Females,Persons,Males,Females,NaN
4,NaN,NaN,NaN,NaN,NaN,1,2,3,4,5,...,32,33,34,35,36,37,38,39,40,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C2308,00,000,INDIA,Total,All ages,1210854977,623270258,587584719,447216165,...,7238719,5354161,1884558,68288971,42120460,26168511,3031570,1647116,1384454,NaN


In [3]:
data.columns.values

array(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4',
       'C-8  EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE - 2011',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45'],
      dtype=object)

In [4]:
data = data.loc[6:,:]
data.drop('Unnamed: 0',axis='columns', inplace=True)
data.drop('Unnamed: 45',axis='columns', inplace=True)
data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,C-8 EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE - 2011,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
6,00,000,INDIA,Total,All ages,1210854977,623270258,587584719,447216165,188506636,...,345724,7238719,5354161,1884558,68288971,42120460,26168511,3031570,1647116,1384454
7,00,000,INDIA,Total,0-6,164515253,85752254,78762999,164515253,85752254,...,0,0,0,0,0,0,0,0,0,0
8,00,000,INDIA,Total,7,24826640,12903364,11923276,6748214,3388683,...,0,0,0,0,0,0,0,136465,75715,60750
9,00,000,INDIA,Total,8,26968373,14061937,12906436,4131414,2013742,...,0,0,0,0,0,0,0,96524,52561,43963
10,00,000,INDIA,Total,9,23424638,12214985,11209653,2491904,1198339,...,0,0,0,0,0,0,0,70452,38456,31996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,65-69,1757,953,804,586,195,...,0,21,14,7,82,72,10,4,1,3
3134,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,70-74,1193,691,502,419,140,...,4,14,9,5,56,41,15,7,4,3
3135,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,75-79,645,343,302,234,80,...,0,6,3,3,22,16,6,7,4,3
3136,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,80+,616,287,329,264,68,...,0,2,1,1,23,14,9,6,1,5


In [5]:
col = ['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Total','Total_male','Total_female','Illiterate','Illiterate_male','Illiterate_female','Literate','Literate_male','Literate_female','Literate without education level','Literate without education level_male','Literate without education level_female','Literate but below primary','Literate but below primary_male','Literate but below primary_female','Primary but below middle','Primary but below middle_male','Primary but below middle_female','Middle but below matric/secondary','Middle but below matric/secondary_male','Middle but below matric/secondary_female','Matric/Secondary but below graduate','Matric/Secondary but below graduate_male','Matric/Secondary but below graduate_female','Higher secondary/Intermediate','Higher secondary/Intermediate_male','Higher secondary/Intermediate_female','Non_Technical','Non_Technical_male','Non_Technical_female','Technical','Technical_male','Technical_female','Graduate and above','Graduate and above_male','Graduate and above_female','Unclassified','Unclassified_male','Unclassified_female']
len(col)

44

In [6]:
literacy_data = data.set_axis(col, axis=1, inplace=False)
literacy_data

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Age-group,Total,Total_male,Total_female,Illiterate,Illiterate_male,...,Non_Technical_female,Technical,Technical_male,Technical_female,Graduate and above,Graduate and above_male,Graduate and above_female,Unclassified,Unclassified_male,Unclassified_female
6,00,000,INDIA,Total,All ages,1210854977,623270258,587584719,447216165,188506636,...,345724,7238719,5354161,1884558,68288971,42120460,26168511,3031570,1647116,1384454
7,00,000,INDIA,Total,0-6,164515253,85752254,78762999,164515253,85752254,...,0,0,0,0,0,0,0,0,0,0
8,00,000,INDIA,Total,7,24826640,12903364,11923276,6748214,3388683,...,0,0,0,0,0,0,0,136465,75715,60750
9,00,000,INDIA,Total,8,26968373,14061937,12906436,4131414,2013742,...,0,0,0,0,0,0,0,96524,52561,43963
10,00,000,INDIA,Total,9,23424638,12214985,11209653,2491904,1198339,...,0,0,0,0,0,0,0,70452,38456,31996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,65-69,1757,953,804,586,195,...,0,21,14,7,82,72,10,4,1,3
3134,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,70-74,1193,691,502,419,140,...,4,14,9,5,56,41,15,7,4,3
3135,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,75-79,645,343,302,234,80,...,0,6,3,3,22,16,6,7,4,3
3136,35,000,ANDAMAN & NICOBAR ISLANDS,Urban,80+,616,287,329,264,68,...,0,2,1,1,23,14,9,6,1,5


In [7]:
df1 = literacy_data.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Total,Total_male,Total_female,Illiterate,Illiterate_male,Illiterate_female,...,Non_Technical_female,Technical,Technical_male,Technical_female,Graduate and above,Graduate and above_male,Graduate and above_female,Unclassified,Unclassified_male,Unclassified_female
Area_Name,,,,,,,,,,,,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,000,Total,All ages,380581,202871,177710,86300,38494,47806,...,121,5138,2935,2203,24159,13622,10537,982,477,505
ANDHRA PRADESH,28,000,Total,All ages,84580777,42442146,42138631,34024017,14190903,19833114,...,21666,705274,587970,117304,5577387,3632622,1944765,349826,197474,152352
ARUNACHAL PRADESH,12,000,Total,All ages,1383727,713912,669815,617722,274044,343678,...,172,4394,3573,821,53606,36430,17176,1874,1112,762
ASSAM,18,000,Total,All ages,31205576,15939443,15266133,12027599,5370804,6656795,...,577,36671,30143,6528,1011345,639241,372104,34022,19944,14078
BIHAR,10,000,Total,All ages,104099452,54278157,49821295,51594899,22670134,28924765,...,34095,35466,30839,4627,3061676,2326092,735584,356290,209443,146847
CHANDIGARH,04,000,Total,All ages,1055450,580663,474787,250012,115317,134695,...,996,13801,9660,4141,194453,104748,89705,1628,801,827
CHHATTISGARH,22,000,Total,All ages,25545198,12832895,12712303,10165276,4025002,6140274,...,1135,51590,45165,6425,1018853,659081,359772,45061,23827,21234
DADRA & NAGAR HAVELI,26,000,Total,All ages,343709,193760,149949,120479,51239,69240,...,69,4748,3645,1103,19174,12227,6947,539,335,204
DAMAN & DIU,25,000,Total,All ages,243247,150301,92946,54841,25658,29183,...,42,2997,2226,771,12802,8425,4377,422,257,165


In [8]:
groups_name = list(df1.groups.keys())
groups_name

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'INDIA',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [9]:
language_data = pd.read_excel("DDW-C19-0000.xlsx", sheet_name='Sheet1')

In [10]:
language_data.head(7)

,"C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Educational level,Number speaking second language,NaN,NaN,Number speaking third language,NaN,NaN
1,code,code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Urban/,NaN,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,Illiterate,42266268,17851584,24414684,3879858,1890285,1989573


In [11]:
language_data = language_data.loc[5:len(language_data)-4,:]

In [12]:
['State_Code','District_Code','Area_Name','Total/Rural/Urban','Educational level','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3']

['State_Code',
 'District_Code',
 'Area_Name',
 'Total/Rural/Urban',
 'Educational level',
 'Number_speaking_second_language',
 'Males2',
 'Females2',
 'Number_speaking_third_language',
 'Males3',
 'Females3']

In [13]:
language_data_total = language_data.set_axis(['State_Code','District_Code','Area_Name','Total/Rural/Urban','Educational level','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3'], axis=1, inplace=False)
language_data_total

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Educational level,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,Illiterate,42266268,17851584,24414684,3879858,1890285,1989573
7,00,00,INDIA,Total,Literate,272722502,158844799,113877703,82129722,48646547,33483175
8,00,00,INDIA,Total,Literate but below primary,29345104,16126959,13218145,3733616,2108024,1625592
9,00,00,INDIA,Total,Primary but below middle,48570544,26588496,21982048,8636296,4782211,3854085
...,...,...,...,...,...,...,...,...,...,...,...
864,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,Literate but below primary,9633,5125,4508,1688,917,771
865,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,Primary but below middle,17565,9892,7673,3395,1905,1490
866,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,Middle but below matric/secondary,19490,11412,8078,5256,3162,2094
867,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,Matric/Secondary but below graduate,33605,18862,14743,14817,8382,6435


### Calculating State Level Data

In [14]:
df1 = language_data_total.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Educational level,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
Area_Name,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,00,Total,Total,257425,142378,115047,72708,41846,30862
ANDHRA PRADESH,28,00,Total,Total,21583334,12193992,9389342,6344376,3836382,2507994
ARUNACHAL PRADESH,12,00,Total,Total,885945,478413,407532,418551,236666,181885
ASSAM,18,00,Total,Total,14460591,7979404,6481187,4352497,2636287,1716210
BIHAR,10,00,Total,Total,16761785,9443495,7318290,1682572,1028475,654097
CHANDIGARH,04,00,Total,Total,579920,322898,257022,321979,178557,143422
CHHATTISGARH,22,00,Total,Total,3384223,1820621,1563602,422569,245150,177419
DADRA & NAGAR HAVELI,26,00,Total,Total,198983,113211,85772,71516,44256,27260
DAMAN & DIU,25,00,Total,Total,116629,71260,45369,34602,21856,12746


In [15]:
group_state = list(df1.groups.keys())
group_state

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'INDIA',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [16]:
df2 = literacy_data.groupby(['Area_Name'])
df2.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Total,Total_male,Total_female,Illiterate,Illiterate_male,Illiterate_female,...,Non_Technical_female,Technical,Technical_male,Technical_female,Graduate and above,Graduate and above_male,Graduate and above_female,Unclassified,Unclassified_male,Unclassified_female
Area_Name,,,,,,,,,,,,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,000,Total,All ages,380581,202871,177710,86300,38494,47806,...,121,5138,2935,2203,24159,13622,10537,982,477,505
ANDHRA PRADESH,28,000,Total,All ages,84580777,42442146,42138631,34024017,14190903,19833114,...,21666,705274,587970,117304,5577387,3632622,1944765,349826,197474,152352
ARUNACHAL PRADESH,12,000,Total,All ages,1383727,713912,669815,617722,274044,343678,...,172,4394,3573,821,53606,36430,17176,1874,1112,762
ASSAM,18,000,Total,All ages,31205576,15939443,15266133,12027599,5370804,6656795,...,577,36671,30143,6528,1011345,639241,372104,34022,19944,14078
BIHAR,10,000,Total,All ages,104099452,54278157,49821295,51594899,22670134,28924765,...,34095,35466,30839,4627,3061676,2326092,735584,356290,209443,146847
CHANDIGARH,04,000,Total,All ages,1055450,580663,474787,250012,115317,134695,...,996,13801,9660,4141,194453,104748,89705,1628,801,827
CHHATTISGARH,22,000,Total,All ages,25545198,12832895,12712303,10165276,4025002,6140274,...,1135,51590,45165,6425,1018853,659081,359772,45061,23827,21234
DADRA & NAGAR HAVELI,26,000,Total,All ages,343709,193760,149949,120479,51239,69240,...,69,4748,3645,1103,19174,12227,6947,539,335,204
DAMAN & DIU,25,000,Total,All ages,243247,150301,92946,54841,25658,29183,...,42,2997,2226,771,12802,8425,4377,422,257,165


### Calculating State Data

In [17]:
state_dict = dict()
country_dict = dict()

In [18]:
temp = df1.get_group(group_state[0])
#temp
list1 = temp['Educational level'].tolist()[1:8]
list1.remove('Literate')
list1

['Illiterate',
 'Literate but below primary',
 'Primary but below middle',
 'Middle but below matric/secondary',
 'Matric/Secondary but below graduate',
 'Graduate and above']

In [19]:
#age_range = {'5-9':['5-9'],'10-14':['10-14'],'15-19':['15-19'],'20-24':['20-24'],'25-29':['25-29'],'30-49':['30-34','35-39','40-44','45-49'],'50-69':['50-54','55-59','60-64','65-69'],'70+':['70-74','75-79','80+']}

In [20]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z[z['Total/Rural/Urban'] == 'Total']
    k = z.iloc[0]
    State = key
    status = 0
    if key=='INDIA':
        status = 1
    #print("State = ",key)
    z2 = df2.get_group(key)
    z2 = z2[z2['Total/Rural/Urban'] == 'Total']
    z2 = z2.iloc[0:1]
    highest_percentage = 0
    for key in list1:
        row1 = z[z['Educational level'] == key]
        Number_speaking_third_language = int(row1['Number_speaking_third_language'])
        population = 0
        if key=='Matric/Secondary but below graduate':  # Find values according to literacy group in Language dataset 
            p1 = int(z2['Matric/Secondary but below graduate'])
            p2 = int(z2['Non_Technical'])
            p3 = int(z2['Technical'])
            p4 = int(z2['Higher secondary/Intermediate'])
            population = p1+p2+p3+p4
        else:
            population = int(z2[key])
            
        percentage = (Number_speaking_third_language*100)/population
        
        if percentage>highest_percentage:
            Group = key
            highest_percentage = percentage
    if status==0:
        #print("Group = ",Group)
        #print("highest_percentage = ",highest_percentage)
        state_dict[State] = [Group,round(highest_percentage,3)]
    else:
        #print("Group = ",Group)
        #print("highest_percentage = ",highest_percentage)
        country_dict[State] = [Group,round(highest_percentage,3)]
    #print("")

In [21]:
state_dict

{'ANDAMAN & NICOBAR ISLANDS': ['Graduate and above', 55.197],
 'ANDHRA PRADESH': ['Graduate and above', 37.073],
 'ARUNACHAL PRADESH': ['Graduate and above', 71.54],
 'ASSAM': ['Graduate and above', 62.465],
 'BIHAR': ['Graduate and above', 8.557],
 'CHANDIGARH': ['Graduate and above', 69.467],
 'CHHATTISGARH': ['Graduate and above', 10.915],
 'DADRA & NAGAR HAVELI': ['Graduate and above', 51.935],
 'DAMAN & DIU': ['Graduate and above', 54.843],
 'GOA': ['Graduate and above', 80.532],
 'GUJARAT': ['Graduate and above', 68.833],
 'HARYANA': ['Graduate and above', 18.04],
 'HIMACHAL PRADESH': ['Graduate and above', 15.25],
 'JAMMU & KASHMIR': ['Graduate and above', 67.385],
 'JHARKHAND': ['Graduate and above', 20.087],
 'KARNATAKA': ['Graduate and above', 48.937],
 'KERALA': ['Graduate and above', 38.702],
 'LAKSHADWEEP': ['Graduate and above', 67.289],
 'MADHYA PRADESH': ['Graduate and above', 11.19],
 'MAHARASHTRA': ['Graduate and above', 64.799],
 'MANIPUR': ['Graduate and above', 64.

In [22]:
a_file = open("literacy-india-b.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state/ut', 'literacy-group', 'percentage'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in state_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [23]:
data = pd.read_csv("literacy-india-b.csv")
data = data.to_string(index=False)
print("Displaying states and union territories of India data")
print("")
print(data)

Displaying states and union territories of India data

                 state/ut     literacy-group  percentage
ANDAMAN & NICOBAR ISLANDS Graduate and above      55.197
           ANDHRA PRADESH Graduate and above      37.073
        ARUNACHAL PRADESH Graduate and above      71.540
                    ASSAM Graduate and above      62.465
                    BIHAR Graduate and above       8.557
               CHANDIGARH Graduate and above      69.467
             CHHATTISGARH Graduate and above      10.915
     DADRA & NAGAR HAVELI Graduate and above      51.935
              DAMAN & DIU Graduate and above      54.843
                      GOA Graduate and above      80.532
                  GUJARAT Graduate and above      68.833
                  HARYANA Graduate and above      18.040
         HIMACHAL PRADESH Graduate and above      15.250
          JAMMU & KASHMIR Graduate and above      67.385
                JHARKHAND Graduate and above      20.087
                KARNATAKA Graduat

In [24]:
country_dict

{'INDIA': ['Graduate and above', 32.431]}

In [25]:
a_file = open("literacy-india-a.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['Country', 'literacy-group', 'percentage'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in country_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [26]:
data = pd.read_csv("literacy-india-a.csv")
data = data.to_string(index=False)
print("Displaying over all country data")
print("")
print(data)

Displaying over all country data

Country     literacy-group  percentage
  INDIA Graduate and above      32.431
